In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('../data/student_performance_data.csv')

In [3]:
def tipifica_variable(df, umbral_categoria=10, umbral_continua=30.0):
    """
    Función para tipificar variables como binaria, categórica, numérica continua y numérica discreta.
    
    Params:
        df: DataFrame de pandas.
        umbral_categoria: Valor entero que define el umbral de la cardinalidad para variables categóricas.
        umbral_continua: Valor flotante que define el umbral de la cardinalidad para variables numéricas continuas.
    
    Returns:
        DataFrame con las columnas (variables) y la tipificación sugerida de cada una.
    """
    
    # Validación del DataFrame
    if not isinstance(df, pd.DataFrame):
        raise ValueError("El argumento 'df' debe ser un DataFrame de pandas válido.")
    
    # Validación de los umbrales
    if not isinstance(umbral_categoria, int) or umbral_categoria <= 0:
        raise ValueError("El 'umbral_categoria' debe ser un número entero mayor que 0.")
    
    if not isinstance(umbral_continua, (float, int)) or umbral_continua <= 0:
        raise ValueError("El 'umbral_continua' debe ser un número flotante mayor que 0.")
    
    # DataFrame inicial con cardinalidad y tipificación sugerida
    df_card = pd.DataFrame({
        "CARD": df.nunique(),
        "%_CARD": df.nunique() / len(df) * 100,
        "tipo_sugerido": ""
    })
    
    # Tipo Binaria
    df_card.loc[df_card["CARD"] == 2, "tipo_sugerido"] = "Binaria"
    
    # Tipo Categórica
    df_card.loc[(df_card["CARD"] < umbral_categoria) & (df_card["tipo_sugerido"] == ""), "tipo_sugerido"] = "Categórica"
    
    # Tipo Numérica Continua
    df_card.loc[(df_card["CARD"] >= umbral_categoria) & (df_card["%_CARD"] >= umbral_continua), "tipo_sugerido"] = "Numerica Continua"
    
    # Tipo Numérica Discreta
    df_card.loc[(df_card["CARD"] >= umbral_categoria) & (df_card["%_CARD"] < umbral_continua), "tipo_sugerido"] = "Numerica Discreta"

    # Selección y renombrado de columnas
    df_card = df_card.reset_index().rename(columns={"index": "nombre_variable"})
    
    return df_card[["nombre_variable", "tipo_sugerido"]]


In [4]:
# Visualizo DF
df.head(5)


,StudentID,Age,Gender,Ethnicity,ParentalEducation,StudyTimeWeekly,Absences,Tutoring,ParentalSupport,Extracurricular,Sports,Music,Volunteering,GPA,GradeClass
0,1001,17,1,0,2,19.833723,7,1,2,0,0,1,0,2.929196,2.0
1,1002,18,0,0,1,15.408756,0,0,1,0,0,0,0,3.042915,1.0
2,1003,15,0,2,3,4.210570,26,0,2,0,0,0,0,0.112602,4.0
3,1004,17,1,0,3,10.028829,14,0,3,1,0,0,0,2.054218,3.0
4,1005,17,1,0,2,4.672495,17,1,3,0,0,0,0,1.288061,4.0


### Usos correctos

In [5]:
# Ejemplo correcto de uso, con valores por defecto
tipifica_variable(df)

,nombre_variable,tipo_sugerido
0,StudentID,Numerica Continua
1,Age,Categórica
2,Gender,Binaria
3,Ethnicity,Categórica
4,ParentalEducation,Categórica
5,StudyTimeWeekly,Numerica Continua
6,Absences,Numerica Discreta
7,Tutoring,Binaria
8,ParentalSupport,Categórica
9,Extracurricular,Binaria


In [6]:
# Ejemplo correcto de uso, con otros valores correctos
tipifica_variable(df, 5, 10)

,nombre_variable,tipo_sugerido
0,StudentID,Numerica Continua
1,Age,Categórica
2,Gender,Binaria
3,Ethnicity,Categórica
4,ParentalEducation,Numerica Discreta
5,StudyTimeWeekly,Numerica Continua
6,Absences,Numerica Discreta
7,Tutoring,Binaria
8,ParentalSupport,Numerica Discreta
9,Extracurricular,Binaria


### Errores

In [7]:
# Ejemplo de errores
tipifica_variable(df, "string", 5)

ValueError: El 'umbral_categoria' debe ser un número entero mayor que 0.

In [30]:
# Ejemplo de errores
tipifica_variable(df, 2.5 , 5)

ValueError: El 'umbral_categoria' debe ser un número entero mayor que 0.

In [31]:
# Ejemplo de errores
tipifica_variable(df, 2 , -5)

ValueError: El 'umbral_continua' debe ser un número flotante mayor que 0.